In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce 
from functions import *
from rapidfuzz import process, fuzz

Load datasets

In [57]:
df_house = pd.read_parquet('datasets/house_post_2014.parquet')

df_dwelling = pd.read_parquet('datasets/mun_dwelling.parquet') # mun_istat_problematic

df_income = pd.read_parquet('datasets/mun_lag_income.parquet') # mun_istat problematic

df_income['year'] = df_income['year'] + 1 # shift birth rate by 1 year to match with house data


df_dem = pd.read_parquet('datasets/mun_dem_no_istat.parquet') # problematic

df_tourism = pd.read_parquet('datasets/mun_tourism.parquet')

# macro

df_imm_emi = pd.read_parquet('datasets/macro/prov_lag_pop_movements.parquet')

df_imm_emi['year'] = df_imm_emi['year'] + 1 # shift birth rate by 1 year to match with house data


df_birth = pd.read_parquet('datasets/macro/lag_birth_rate.parquet')

df_birth['year'] = df_birth['year'] + 1 # shift birth rate by 1 year to match with house data


df_bond = pd.read_parquet('datasets/macro/lag_bond.parquet')

df_bond['year'] = df_bond['year'] + 1 # shift birth rate by 1 year to match with house data


df_cpi = pd.read_parquet('datasets/macro/lag_cpi.parquet')

df_cpi['year'] = df_cpi['year'] + 1 # shift birth rate by 1 year to match with house data


df_gini = pd.read_parquet('datasets/macro/lag_gini.parquet')

df_gini['year'] = df_gini['year'] + 1 # shift birth rate by 1 year to match with house data


df_life = pd.read_parquet('datasets/macro/lag_life.parquet')

df_life['year'] = df_life['year'] + 1 # shift birth rate by 1 year to match with house data


df_real_gdp = pd.read_parquet('datasets/macro/lag_real_gdp.parquet')

df_real_gdp['year'] = df_real_gdp['year'] + 1 # shift birth rate by 1 year to match with house data


df_unemployment = pd.read_parquet('datasets/macro/lag_unemployment.parquet')

df_unemployment['year'] = df_unemployment['year'] + 1 # shift birth rate by 1 year to match with house data


In [58]:
# istat codes updated to 2025
df_new_istat = pd.read_parquet('datasets/mun_istat_codes.parquet')

# istat codes changes
df_change = pd.read_parquet('datasets/changes_istat.parquet')

In [59]:
df_new_istat = df_new_istat[['mun_istat','mun_name_norm']]

1. update istat codes
2. check for istat codes that have not been changed -> manually check
3. merge

df_house

In [60]:
df_house_updated = update_istat(
    df=df_house,
    df_map=df_change, 
    valid_codes=df_new_istat["mun_istat"], 
    istat_col="mun_istat",
    istat_old = "mun_istat_old",
    istat_new = "mun_istat_new"
)

df_house_updated = df_house_updated.drop(columns = 'mun_istat')

In [61]:
# split in suppressed (no correspondance with latest istat codes) and non suppressed
suppressed_df = df_house_updated[df_house_updated['suppressed'] == True].copy()
non_suppressed_df = df_house_updated[df_house_updated['suppressed'] == False].copy()

In [62]:
# check for similarity in unmatched mun_name_norm (suppressed)
similarity = similarity_score(suppressed_df, df_new_istat, col = 'mun_name_norm')
similarity

,Name in df1,Name in df2,Similarity score (0-100)
134,loiri porto s paolo,loiri porto san paolo,95.000000
10,castellar,castellaro,94.736842
132,trinita agultu vignola,trinita dagultu e vignola,93.617021
73,vezzano,avezzano,93.333333
12,veruno,verduno,92.307692
...,...,...,...
78,zambana,agna,77.142857
120,acquacanina,acqualagna,76.190476
23,valsecca,valmacca,75.000000
79,zuclo,zuglio,72.727273


In [63]:
# manually sobstitute non-corresponding mun names
suppressed_df['mun_name_norm'] = suppressed_df['mun_name_norm'].replace({
    'loiri porto s paolo':	'loiri porto san paolo',
'castellar' :	'castellaro',
'trinita agultu vignola':	'trinita dagultu e vignola',
's antonio di gallura':	'santantonio di gallura',
'tonengo' :	'moransengotonengo',
'piovera':	'alluvioni piovera',
'quaregna'	:'quaregna cerreto',
'cellio' :	'cellio con breia',
'lisignago' :	'cembra lisignago',
'condino':  	'castel condino',
'osmate'	: 'cadrezzate con osmate',
'veddasca' 	:'maccagno con pino e veddasca',
'malgesso'	:'bardello con malgesso e bregano',
'villa vicentina'	:'fiumicello villa vicentina',
'lusiana'	:'lusiana conco',
'vigolo vattaro'	:'vigolo',
'gravedona'	:'gravedona ed uniti',
'sorbolo'	:'sorbolo mezzani',
'san marcello pistoiese'	:'san marcello',
'presicce'	:'presicceacquarica',
'rossano'	:'rossano veneto',
'ripe'	:'ripe san ginesio',
'cutigliano'	:'abetone cutigliano',
'lorenzana'	:'crespina lorenzana',
'sillano'	:'sillano giuncugnano',
'zibello'	:'polesine zibello',
'piandisco':	'castelfranco piandisco',
'mossano'	 : 'barbarano mossano'
})

In [64]:
suppressed_df = pd.merge(suppressed_df, df_new_istat, on = ['mun_name_norm'], how = 'left')

# check number of umatched cases
unmatched = suppressed_df[suppressed_df["mun_istat"].isna()]
print("Unmatched names:", unmatched["mun_name_norm"].nunique())

# drop rows where new ISTAT could not be found
suppressed_df = suppressed_df[suppressed_df['mun_istat'].notna()]

# replace ISTAT code in suppressed_df
suppressed_df['mun_istat_updated'] = suppressed_df['mun_istat']

# drop mun_istat
suppressed_df = suppressed_df.drop(columns=['mun_istat'])

# concatenate with non-suppressed rows
df_house_updated = pd.concat([non_suppressed_df, suppressed_df], ignore_index=True)

df_house_updated = df_house_updated.drop(columns = ['changed','suppressed'])

Unmatched names: 108


In [65]:
# group by ['mun_istat','year','sector','type','condition']
df_house_updated = df_house_updated.groupby(
        ['mun_istat_updated','year','sector','type','condition'],
        observed=True              
    ).agg({
    # keep one representative name
    'mun_key': 'first', 
    'mun_name': 'first', 
    'mun_name_norm': 'first',
    'region': 'first',
    'prov' : 'first',
    'log_buy_min' : 'mean',
    'log_buy_max' : 'mean',
    'log_buy_avg': 'mean'
}).reset_index()

df_house_updated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549552 entries, 0 to 549551
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   mun_istat_updated  549552 non-null  object  
 1   year               549552 non-null  int64   
 2   sector             549552 non-null  category
 3   type               549552 non-null  object  
 4   condition          549552 non-null  category
 5   mun_key            549552 non-null  object  
 6   mun_name           549552 non-null  object  
 7   mun_name_norm      549552 non-null  object  
 8   region             549552 non-null  category
 9   prov               549552 non-null  object  
 10  log_buy_min        549552 non-null  float64 
 11  log_buy_max        549552 non-null  float64 
 12  log_buy_avg        549552 non-null  float64 
dtypes: category(3), float64(3), int64(1), object(6)
memory usage: 43.5+ MB


df_dwelling

In [66]:
df_dwelling_updated = update_istat(
    df=df_dwelling,
    df_map=df_change, 
    valid_codes=df_new_istat["mun_istat"], 
    istat_col="mun_istat",
    istat_old = "mun_istat_old",
    istat_new = "mun_istat_new"
)

df_dwelling_updated = df_dwelling_updated.drop(columns = ['mun_istat','occupation'])

df_dwelling_updated['mun_name_norm'] = df_dwelling_updated['mun_name'].apply(normalize_name)

In [67]:
# split in suppressed (no correspondance with latest istat codes) and non suppressed
suppressed_df = df_dwelling_updated[df_dwelling_updated['suppressed'] == True].copy()
non_suppressed_df = df_dwelling_updated[df_dwelling_updated['suppressed'] == False].copy()

In [68]:
similarity = similarity_score(suppressed_df, df_new_istat, col = 'mun_name_norm')
similarity

,Name in df1,Name in df2,Similarity score (0-100)
0,malgesso,bardello con malgesso e bregano,90.0
1,tonengo,moransengotonengo,90.0
2,uggiatetrevano,re,90.0
3,vighizzolo deste,este,90.0
4,quero vas,crova,72.0


In [69]:
# manually sobstitute non-corresponding mun_names
suppressed_df['mun_name_norm'] = suppressed_df['mun_name_norm'].replace({
    'malgesso' : 'bardello con malgesso e bregano',
    'tonengo' : 'moransengotonengo'
})

suppressed_df = pd.merge(suppressed_df, df_new_istat, on = ['mun_name_norm'], how = 'left')

# see unmatched (without ISTAT)
unmatched = suppressed_df[suppressed_df["mun_istat"].isna()]
print("Unmatched names:", unmatched["mun_name_norm"].nunique())

# drop rows where new ISTAT could not be found
suppressed_df = suppressed_df[suppressed_df['mun_istat'].notna()]

# replace ISTAT code in suppressed_df
suppressed_df['mun_istat_updated'] = suppressed_df['mun_istat']

# drop mun_istat
suppressed_df = suppressed_df.drop(columns=['mun_istat'])

# concatenate with non-suppressed rows
df_dwelling_updated = pd.concat([non_suppressed_df, suppressed_df], ignore_index=True)

df_dwelling_updated = df_dwelling_updated.drop(columns = ['changed','suppressed'])

Unmatched names: 3


df_tourism

In [70]:
df_tourism_updated = update_istat(
    df=df_tourism,
    df_map=df_change, 
    valid_codes=df_new_istat["mun_istat"], 
    istat_col="mun_istat",
    istat_old = "mun_istat_old",
    istat_new = "mun_istat_new"
)

df_tourism_updated['mun_name_norm'] = df_tourism_updated['mun_name'].apply(normalize_name)

df_tourism_updated = df_tourism_updated.drop(columns = ['mun_istat','region','mun_name'])

In [71]:
suppressed_df = df_tourism_updated[df_tourism_updated['suppressed'] == True].copy()
non_suppressed_df = df_tourism_updated[df_tourism_updated['suppressed'] == False].copy()

similarity = similarity_score(suppressed_df, df_new_istat, col = 'mun_name_norm')
similarity

,Name in df1,Name in df2,Similarity score (0-100)
25,castellar,castellaro,94.736842
92,vezzano,avezzano,93.333333
23,veruno,verduno,92.307692
13,varena,varenna,92.307692
90,tuenno,tenno,90.909091
...,...,...,...
53,cavacurta,cave,77.142857
73,acquacanina,acqualagna,76.190476
106,valsecca,valmacca,75.000000
96,zuclo,zuglio,72.727273


In [72]:
suppressed_df['mun_name_norm'] = suppressed_df['mun_name_norm'].replace({
    'tonengo':	'moransengotonengo',
'malgesso':	'bardello con malgesso e bregano',
'osmate':	'cadrezzate con osmate',
'piovera':	'alluvioni piovera',
'cellio':	'cellio con breia',
'quaregna':	'quaregna cerreto',
'piadena':	'piadena drizzona',
'sorbolo': 'sorbolo mezzani',
'villa vicentina':	'fiumicello villa vicentina',
'mossano':	'barbarano mossano',
'lusiana':	'lusiana conco',
'presicce':	'presicceacquarica',
'cornale':	'cornale e bastida',
'veddasca':	'maccagno con pino e veddasca',
'sillano':	'sillano giuncugnano',
'zibello':	'polesine zibello',
'cutigliano':	'abetone cutigliano',
'lisignago':	'cembra lisignago'
})

suppressed_df = pd.merge(suppressed_df, df_new_istat, on = ['mun_name_norm'], how = 'left')

# see unmatched (without ISTAT)
unmatched = suppressed_df[suppressed_df["mun_istat"].isna()]
print("Unmatched names:", unmatched["mun_name_norm"].nunique())

# drop rows where new ISTAT could not be found
suppressed_df = suppressed_df[suppressed_df['mun_istat'].notna()]

# replace ISTAT code in suppressed_df
suppressed_df['mun_istat_updated'] = suppressed_df['mun_istat']

# drop mun_istat
suppressed_df = suppressed_df.drop(columns=['mun_istat'])

# concatenate with non-suppressed rows
df_tourism_updated = pd.concat([non_suppressed_df, suppressed_df], ignore_index=True)

df_tourism_updated = df_tourism_updated.drop(columns = ['changed','suppressed'])

Unmatched names: 97


df_dem

In [73]:
df_dem_updated = update_istat(
    df=df_dem,
    df_map=df_change, 
    valid_codes=df_new_istat["mun_istat"], 
    istat_col="mun_istat",
    istat_old = "mun_istat_old",
    istat_new = "mun_istat_new"
)

df_dem_updated = df_dem_updated.drop(columns = ['mun_istat','prov_name'])

In [74]:
suppressed_df = df_dem_updated[df_dem_updated['suppressed'] == True].copy()
non_suppressed_df = df_dem_updated[df_dem_updated['suppressed'] == False].copy()

similarity = similarity_score(suppressed_df, df_new_istat, col = 'mun_name_norm')
similarity

,Name in df1,Name in df2,Similarity score (0-100)
11,san floriano del collioteverjan,san floriano del colliosteverjan,98.412698
12,savogna disonzosovodnje ob soi,savogna disonzosovodnje ob soci,98.360656
2,duino aurisinadevin nabreina,duino aurisinadevin nabrezina,98.245614
8,pontcanavese,pont canavese,96.000000
3,grana,monterosso grana,90.000000
0,campospinoso,campospinoso albaredo,90.000000
1,casorzo,casorzo monferrato,90.000000
14,tripi,tripi abakainon,90.000000
6,montemagno,montemagno monferrato,90.000000
5,montagnamontan,monta,90.000000


In [75]:
suppressed_df['mun_name_norm'] = suppressed_df['mun_name_norm'].replace({
    'san floriano del collioteverjan':	'san floriano del colliosteverjan',
'savogna disonzosovodnje ob soi':	'savogna disonzosovodnje ob soci',
'duino aurisinadevin nabreina':	'duino aurisinadevin nabrezina',
'pontcanavese':	'pont canavese',
'grana':	'monterosso grana',
'campospinoso':	'campospinoso albaredo',
'casorzo':	'casorzo monferrato',
'tripi':	'tripi  abakainon',
'montemagno':	'montemagno monferrato',
'ionadi':	'jonadi'
})

suppressed_df = pd.merge(suppressed_df, df_new_istat, on = ['mun_name_norm'], how = 'left')

# see unmatched (without ISTAT)
unmatched = suppressed_df[suppressed_df["mun_istat"].isna()]
print("Unmatched names:", unmatched["mun_name_norm"].nunique())

# drop rows where new ISTAT could not be found
suppressed_df = suppressed_df[suppressed_df['mun_istat'].notna()]

# replace ISTAT code in suppressed_df
suppressed_df['mun_istat_updated'] = suppressed_df['mun_istat']

# drop mun_istat
suppressed_df = suppressed_df.drop(columns=['mun_istat'])

# concatenate with non-suppressed rows
df_dem_updated = pd.concat([non_suppressed_df, suppressed_df], ignore_index=True)

df_dem_updated = df_dem_updated.drop(columns = ['changed','suppressed','mun_name_norm'])

Unmatched names: 5


df_income (no mun_name_norm)

In [76]:
df_income_updated = update_istat(
    df=df_income,
    df_map=df_change, 
    valid_codes=df_new_istat["mun_istat"], 
    istat_col="mun_istat",
    istat_old = "mun_istat_old",
    istat_new = "mun_istat_new"
)

df_income_updated = df_income_updated.drop(columns = ['mun_istat'])

In [77]:
# print columns were suppressed is true and check istat codes
suppressed_df = df_income_updated[df_income_updated['suppressed'] == True].copy()

In [78]:
# manually sobstitute istat codes
df_income_updated['mun_istat_updated'] = df_income_updated['mun_istat_updated'].replace({
    '090001':	'113001',
'090002':	'113003',
'090006':	'113004',
'090009':	'113006',
'090014':	'113007',
'090017':	'113008',
'090021':	'113010',
'090035':	'113012',
'090036':	'113014',
'090037':	'113015',
'090041':	'113016',
'090044':	'112034',
'090047':	'113017',
'090049':	'113018',
'090054':	'113020',
'090062':	'113002',
'090063':	'113022',
'090070':	'113025',
'090074':	'113026',
'090080':	'113024',
'090081':	'113005',
'090083':	'113011',
'090084':	'113013',
'090085':	'113023',
'090090':	'113019',
'091002':	'116001',
'091005':	'116002',
'091006':	'116003',
'091019':	'116005',
'091026':	'116006',
'091031':	'116007',
'091032':	'116008',
'091035':	'116009',
'091037':	'116010',
'091039':	'116011',
'091042':	'116012',
'091069':	'116013',
'091072':	'116014',
'091088':	'116015',
'091089':	'116016',
'091095':	'116017',
'091097':	'116018',
'091098':	'116019',
'091099':	'116020',
'091100':	'116021',
'091101':	'116022',
'091103':	'116004',
'092009':	'118006',
'092109':	'118007',
'095006':	'115006',
'095037':	'115044',
'095065':	'115075',
'095067':	'115078',
'095083':	'115029',
'097080':	'016215',
'104001':	'113001',
'104002':	'113002',
'104003':	'113003',
'104004':	'113004',
'104005':	'113005',
'104006':	'113006',
'104007':	'113007',
'104009':	'113008',
'104010':	'113010',
'104011':	'113011',
'104012':	'113012',
'104013':	'113013',
'104014':	'113014',
'104015':	'113015',
'104016':	'113016',
'104017':	'113017',
'104018':	'113018',
'104020':	'113020',
'104021':	'113023',
'104022':	'113022',
'104023':	'113021',
'104024':	'113024',
'104025':	'113025',
'104026':	'113026',
'105001':	'116001',
'105002':	'116002',
'105003':	'116003',
'105004':	'116004',
'105005':	'116005',
'105006':  '116006',
'105007':	'116007',
'105008':	'116008',
'105009':	'116009',
'105010':	'116010',
'105011':	'116011',
'105012':	'116012',
'105013':	'116013',
'105014':	'116014',
'105016':	'116015',
'105017':	'116016',
'105018':	'116017',
'105019':	'116018',
'105020':	'116019',
'105021':	'116020',
'105022':	'116021',
'105023':	'116022'
})


Merge

In [79]:
# merge national house, income, demographic, and tourism data on [mun_istat_updated, year]
dfs1 = [df_house_updated, df_dem_updated, df_income_updated, df_tourism_updated]

df = reduce(lambda left, right: pd.merge(left, right, on = ['mun_istat_updated','year'], how = 'left'), dfs1)

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 554858 entries, 0 to 554857
Data columns (total 35 columns):
 #   Column                   Non-Null Count   Dtype   
---  ------                   --------------   -----   
 0   mun_istat_updated        554858 non-null  object  
 1   year                     554858 non-null  int64   
 2   sector                   554858 non-null  category
 3   type                     554858 non-null  object  
 4   condition                554858 non-null  category
 5   mun_key                  554858 non-null  object  
 6   mun_name                 554858 non-null  object  
 7   mun_name_norm_x          554858 non-null  object  
 8   region                   554858 non-null  category
 9   prov                     554858 non-null  object  
 10  log_buy_min              554858 non-null  float64 
 11  log_buy_max              554858 non-null  float64 
 12  log_buy_avg              554858 non-null  float64 
 13  population               553303 non-null  fl

In [81]:
# merge dwellings count on [mun_istat]
df = pd.merge(df, df_dwelling_updated, on = ['mun_istat_updated'], how = 'left')

In [82]:
# drop null values
df = df.dropna()

Merge macros

In [83]:
# merge df_real_gdp, df_cpi. df_gini, df_unemployment, df_birth, df_life on [year]
dfs2 = [df, df_real_gdp, df_cpi, df_gini, df_unemployment, df_birth, df_life]

df = reduce(lambda left, right: pd.merge(left, right, on = ['year'], how = 'left'), dfs2)

In [84]:
# merge df_imm_emi on [prov, year]
df = pd.merge(df, df_imm_emi, on = ['prov', 'year'], how = 'left')

In [85]:
print('Number data lost from from df_house:', len(df_house_updated) - len(df))
print('Percentage: ', ((len(df_house_updated) - len(df))/len(df_house)) * 100)

Number data lost from from df_house: -4437
Percentage:  -0.3259865740846184


In [86]:
df = df.rename(columns = {
    'mun_istat_updated' : 'mun_istat'
})

Save dataset

In [87]:
df.to_parquet('datasets/pre_processing.parquet', index = False)